In [1]:
import os
import process_bigraph as pbg
from vivarium.vivarium import Vivarium
import numpy as np

from wholecell.utils import units
from ecoli.shared.schemas import numpy_schema, listener_schema, get_config_schema
from ecoli import ecoli_core
from ecoli.shared.base import ProcessBase

from bigraph_schema.type_system import TypeSystem

<unknown>:1: SyntaxWarning: invalid escape sequence '\s'


In [2]:
core = ecoli_core

states = []

class Func(pbg.Process):
    config_schema = {
        'k': 'float',
        'n_rna': 'maybe[integer]'
    }

    def __init__(self, config=None, core=None):
        super().__init__(config, core)
        self.k = self.config.get('k', np.random.random())
        self.n_rna = np.random.randint(1, 11)

        # passing an initial state list, but parsing and returning as an array
        self.generate = lambda: np.random.random((self.n_rna,))
        self.rna_type = {"_type": "list", "_default": self.generate()}

    def initial_state(self):
        return {
            # bulk molecules
            "listeners": {
                "replication_data": listener_schema(
                    {"i": np.random.random(), "j": np.random.random()}
                ),
                "rRNA": self.generate(),
            },
        }
            

    def inputs(self):
        return {
            # bulk molecules
            "bulk": numpy_schema("bulk"),
            "listeners": {
                "mass": listener_schema({"cell": np.random.random()}),
                "replication_data": listener_schema(
                    {"i": np.random.random(), "j": np.random.random()}
                ),
                "rRNA": self.rna_type,
            },
            "environment": {
                "media_id": "string"
            },
            
        }

    def outputs(self):
        return {
            # bulk molecules
            "listeners": {
                "replication_data": listener_schema(
                    {"i": np.random.random(), "j": np.random.random()}
                ),
                "rRNA": self.rna_type,
            }
        }

    def update(self, state, interval):
        listeners = state.get('listeners')
        replication_data = listeners.get('replication_data')
        x = listeners.get('mass').get('cell') * (replication_data.get('i') / replication_data.get('j'))
        y = (x**x) * interval
        rRNA_k = np.array(state.get('listeners').get('rRNA') * round(y))
        return {
            'listeners': {
                'replication_data': {
                    'i': y * replication_data.get('i'),
                    'j': y * replication_data.get('j')
                },
                'rRNA': rRNA_k
            },
            
        }

core.process_registry.register('func', Func)

In [4]:
from vivarium.vivarium import Vivarium


def vivarium(core, doc=None):
    return Vivarium(
        core=core, processes=core.process_registry.registry, types=core.types(), document=doc
    )

doc = {
    'state': {
        'func': {
            '_type': 'process',
            'address': 'local:func',
            'inputs': {
                'bulk': ['bulk'],
                'listeners': ['listeners'],
                'environment': ['environment'],
            },
            'outputs': {
                'listeners': ['listeners'],
            },
        }
    }
}

viv = vivarium(core, doc)
viv.add_emitter()
dur = 11

import json 

# with open('./test_document.json', 'w') as f:
#     json.dump(viv.make_document())
# results = []
# for i in np.random.random((dur,)):
#     viv.run(float(i))
#     results.append({'duration': i, 'data': viv.get_results()[-1]})
viv

Vivarium( 
{ 'bulk': [],
  'emitter': { 'address': 'local:ram-emitter',
               'config': { 'emit': { 'bulk': 'any',
                                     'environment': 'any',
                                     'global_time': 'any',
                                     'listeners': 'any'}},
               'inputs': { 'bulk': ['bulk'],
                           'environment': ['environment'],
                           'global_time': ['global_time'],
                           'listeners': ['listeners']},
               'instance': <process_bigraph.emitter.RAMEmitter object at 0x12a6df2c0>,
               'outputs': None},
  'environment': {'media_id': ''},
  'func': { 'address': 'local:func',
            'config': {'k': 0.0, 'n_rna': None},
            'inputs': { 'bulk': ['bulk'],
                        'environment': ['environment'],
                        'listeners': ['listeners']},
            'instance': <__main__.Func object at 0x12bb2b1d0>,
            'interval': 1

In [5]:
import json

In [7]:
viv.make_document()

Exception: no method "serialize" found for state [] and schema {'_type': 'list[tuple]', '_default': [], '_check': 'check_list', '_slice': 'slice_list', '_apply': <function bulk_numpy_updater at 0x12b925ee0>, '_serialize': <class 'ecoli.library.schema.get_bulk_counts'>, '_deserialize': 'deserialize_list', '_dataclass': 'dataclass_list', '_fold': 'fold_list', '_divide': <function divide_bulk at 0x12b926340>, '_type_parameters': ['element'], '_description': 'general list type (or sublists)', '_element': {'_type': 'tuple', '_default': 'default_tuple', '_apply': 'apply_tuple', '_check': 'check_tuple', '_slice': 'slice_tuple', '_serialize': 'serialize_tuple', '_deserialize': 'deserialize_tuple', '_dataclass': 'dataclass_tuple', '_fold': 'fold_tuple', '_divide': 'divide_tuple', '_bind': 'bind_tuple', '_description': 'tuple of an ordered set of typed values'}}

In [6]:
viv.get_results()

[{'global_time': 0.0,
  'bulk': [],
  'listeners': {'replication_data': {'i': 0.4806334555195192,
    'j': 0.05948634812681208},
   'rRNA': array([0.59227979, 0.630577  , 0.3686462 , 0.16954703, 0.66315774,
          0.91253483]),
   'mass': {'cell': 0.6894568486785687}},
  'environment': {'media_id': ''},
  'timestep': 0.0},
 {'global_time': 1.0,
  'bulk': [],
  'listeners': {'replication_data': {'i': 6870.5773753302365,
    'j': 850.347708607428},
   'rRNA': array([ 8466.63964896,  9014.09823583,  5269.79739071,  2423.67481788,
           9479.8399548 , 13044.68544279]),
   'mass': {'cell': 0.6894568486785687}},
  'environment': {'media_id': ''},
  'timestep': 1.0},
 {'global_time': 2.0,
  'bulk': [],
  'listeners': {'replication_data': {'i': 98213790.42242445,
    'j': 12155582.722819502},
   'rRNA': array([1.21030614e+08, 1.28856534e+08, 7.53317537e+07, 3.46464315e+07,
          1.35514312e+08, 1.86473778e+08]),
   'mass': {'cell': 0.6894568486785687}},
  'environment': {'media_id'

In [7]:
viv.get_timeseries()

,global_time,bulk,listeners.replication_data.i,listeners.replication_data.j,listeners.rRNA,listeners.mass.cell,environment.media_id,timestep
0,0.0,[],4.806335e-01,5.948635e-02,"[0.5922797935615909, 0.6305770014570625, 0.368...",0.689457,,0.0
1,1.0,[],6.870577e+03,8.503477e+02,"[8466.639648962942, 9014.09823582871, 5269.797...",0.689457,,1.0
2,2.0,[],9.821379e+07,1.215558e+07,"[121030613.78192526, 128856534.2811714, 753317...",0.689457,,2.0
3,3.0,[],1.403950e+12,1.737621e+11,"[1730132624012.6216, 1842004157549.3452, 10768...",0.689457,,3.0
4,4.0,[],2.006924e+16,2.483901e+15,"[2.4732245860260424e+16, 2.633144943216789e+16...",0.689457,,4.0
5,5.0,[],2.868865e+20,3.550696e+19,"[3.5354745457242276e+20, 3.764080696328399e+20...",0.689457,,5.0


In [8]:
from ecoli.experiments.ecoli_master_sim import SimConfig

In [19]:
config = SimConfig()

In [21]:
dir(config)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_config',
 'default_config_path',
 'get',
 'keys',
 'merge_config_dicts',
 'parser',
 'to_dict',
 'update_from_cli',
 'update_from_dict',
 'update_from_json']

In [16]:
n = np.random.randint(1,11)
x = np.zeros((n,))

In [9]:
results

[{'duration': np.float64(0.5882809324478524),
  'data': {'global_time': 0.0,
   'bulk': [],
   'listeners': {'replication_data': {'i': 0.4806334555195192,
     'j': 0.05948634812681208},
    'rRNA': array([0.59227979, 0.630577  , 0.3686462 , 0.16954703, 0.66315774,
           0.91253483]),
    'mass': {'cell': 0.6894568486785687}},
   'environment': {'media_id': ''},
   'timestep': 0.0}},
 {'duration': np.float64(0.6205691171702246),
  'data': {'global_time': 1.0,
   'bulk': [],
   'listeners': {'replication_data': {'i': 6870.5773753302365,
     'j': 850.347708607428},
    'rRNA': array([ 8466.63964896,  9014.09823583,  5269.79739071,  2423.67481788,
            9479.8399548 , 13044.68544279]),
    'mass': {'cell': 0.6894568486785687}},
   'environment': {'media_id': ''},
   'timestep': 1.0}},
 {'duration': np.float64(0.8068815672193903),
  'data': {'global_time': 2.0,
   'bulk': [],
   'listeners': {'replication_data': {'i': 98213790.42242445,
     'j': 12155582.722819502},
    'rRNA'

In [8]:
import numpy as np
coords = []
getCoords = ((
    map(
        lambda v: v**v / v,
        np.random.random((11,)).tolist()
    )
))

In [9]:
for _ in range(11):
    val_i = next(getCoords)
    print(val_i)

2.491581280065981
25.99649272137472
1.00884109254635
1.1895802899347374
1.9358864184890763
1.0188348334522326
3.9796164211391187
1.2880151138688403
1.348989908171287
1.1793719828385398
7.137569204168612


In [23]:
class Data:
    def __init__(self):
        self.corpus = np.random.random(self.shape).tolist()
        self.value_iterator = iter(self.corpus)

    @property
    def shape(self):
        bounds = (11, 22)
        generate = lambda: np.random.randint(*bounds)
        return (generate(), generate())
    
    @property
    def value(self):
        return next(self.value_iterator)

    def __next__(self):
        return next(self.value_iterator)

In [24]:
d = Data()

In [32]:
class ResponseData:
    def __

False

In [40]:
import dataclasses
import pickle
import uuid
import fastapi
from fastapi import FastAPI, BackgroundTasks, Request
from asyncio import Queue
import asyncio

from fastapi.datastructures import QueryParams
from vivarium.vivarium import Vivarium

In [41]:
params = QueryParams(dict(zip(['x', 'y', 'z'], [11.11, 22.2, 11.22])))

In [42]:
eval(params.get('x'))

11.11

In [45]:
r = Request({'type': 'http'}, query_)

In [11]:
get_times = lambda dur, dt: np.arange(0, dur, dt).tolist()

In [12]:
dur = 11
dt = 0.00022
times = get_times(dur, dt)

In [ ]:
times

In [9]:
from process_bigraph import pp, ProcessTypes

In [10]:
from api.tests.test_processes import Func, SimpleTest
core = ProcessTypes()
core.process_registry.register('func', Func)
core.process_registry.register('simple-test', SimpleTest)

In [15]:
from vivarium.vivarium import Vivarium

doc = {
        'state': {
            'x': {
                '_type': 'process',
                'address': 'local:simple-test',
                'inputs': {
                    'x': ['x'],
                    'y': ['y'],
                },
                'outputs': {
                    'x': ['x'],
                    'y': ['y'],
                },
            }
        }
    }

doc2 = {
    "state": {
        "global_time": "0.0",
        "func": {
            "inputs": {
                "bulk": [
                    "bulk"
                ],
                "listeners": [
                    "listeners"
                ],
                "environment": [
                    "environment"
                ]
            },
            "outputs": {
                "listeners": [
                    "listeners"
                ]
            },
            "interval": 1.0,
            "address": "local:func",
            "config": {
                "k": "0.0",
                "n_rna": "!nil"
            },
            "shared": {}
        },
        "bulk": [
            []
        ],
        "listeners": {
            "mass": {
                "cell": "0.9154434399648245"
            },
            "replication_data": {
                "i": "0.5463138160228774",
                "j": "0.73607425441701"
            },
            "rRNA": [
                "0.6695704180720157",
                "0.1058517535813428",
                "0.7796836175728571",
                "0.033221210885723274",
                "0.011709351683637514",
                "0.9901155071921232",
                "0.5992292173977998",
                "0.37967558092486176"
            ]
        },
        "environment": {
            "media_id": ""
        },
        "emitter": {
            "address": "local:ram-emitter",
            "config": {
                "emit": {
                    "global_time": "any",
                    "bulk": "any",
                    "listeners": "any",
                    "environment": "any"
                }
            },
            "inputs": {
                "global_time": [
                    "global_time"
                ],
                "bulk": [
                    "bulk"
                ],
                "listeners": [
                    "listeners"
                ],
                "environment": [
                    "environment"
                ]
            },
            "outputs": null
        }
    },
    "composition": "(global_time:float|func:process[(bulk:list[tuple]|listeners:(mass:(cell:float)|replication_data:(i:float|j:float)|rRNA:list)|environment:(media_id:string)),(listeners:(replication_data:(i:float|j:float)|rRNA:list))]|bulk:list[tuple][()]|listeners:(mass:(cell:float)|replication_data:(i:float|j:float)|rRNA:list[()])|environment:(media_id:string)|emitter:step[(global_time:any|bulk:any|listeners:any|environment:any),()])"
}


v = Vivarium(core=core, types=core.types(), processes=core.process_registry.registry, document=doc2)

NameError: name 'null' is not defined

In [12]:
v.add_process('simple', 'simple-test', inputs={'x': ['x'], 'y': ['y']})

In [13]:
v

Vivarium( 
{ 'global_time': 0.0,
  'simple': { 'address': 'local:simple-test',
              'config': {},
              'inputs': {'x': ['x'], 'y': ['y']},
              'instance': <api.tests.test_processes.SimpleTest object at 0x12e63f290>,
              'interval': 1.0,
              'outputs': {},
              'shared': None},
  'x': 0.0,
  'y': 0.0})